# Leer los datos

In [9]:
import pandas as pd
import sys
from itertools import combinations
from pprint import pprint

# Aumentamos el límite de recursión porque el backtracking puede ser muy profundo
sys.setrecursionlimit(10000)

ARCHIVO = "datos.csv"
TOMAS = 30
ACTORES = 10

df = pd.read_csv(ARCHIVO)

df.head()

,Toma,1,2,3,4,5,6,7,8,9,10,Unnamed: 11,Total
0,1,1,1,1,1,1,0,0,0,0,0,NaN,5
1,2,0,0,1,1,1,0,0,0,0,0,NaN,3
2,3,0,1,0,0,1,0,1,0,0,0,NaN,3
3,4,1,1,0,0,0,0,1,1,0,0,NaN,4
4,5,0,1,0,1,0,0,0,1,0,0,NaN,3


In [16]:
actores = [str(idx) for idx in range(1, ACTORES+1)]
tomas = list(range(1, TOMAS+1))
actores_en_toma = dict()

for _, row in df.iterrows():
    t = int(row['Toma'])
    l_actores = [int(col) for col in actores if int(row[col]) == 1]
    actores_en_toma[t] = l_actores

print("Actores:", actores)
print("Tomas:", tomas)
print("Actores en toma:")
pprint(actores_en_toma)


Actores: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
Tomas: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Actores en toma:
{1: [1, 2, 3, 4, 5],
 2: [3, 4, 5],
 3: [2, 5, 7],
 4: [1, 2, 7, 8],
 5: [2, 4, 8],
 6: [1, 2, 4, 5],
 7: [1, 2, 4, 5],
 8: [1, 2, 6],
 9: [1, 2, 4],
 10: [1, 2, 6, 9],
 11: [1, 2, 3, 5, 8],
 12: [1, 2, 3, 4, 6],
 13: [1, 4, 5],
 14: [1, 3, 6],
 15: [1, 2, 7],
 16: [4, 10],
 17: [1, 3],
 18: [3, 6],
 19: [1, 3],
 20: [1, 3, 4, 5],
 21: [6, 8],
 22: [1, 2, 3, 4],
 23: [1, 3],
 24: [3, 6],
 25: [1, 2, 4, 10],
 26: [1, 3, 5, 9],
 27: [4, 5],
 28: [1, 4],
 29: [1, 5, 6],
 30: [1, 4]}
